In [ ]:

import os
import yaml
import torch
import random
import logging
import statistics
import numpy as np
from glob import glob
from tqdm import tqdm
import src.clip as clip
from torchinfo import summary
import torch.nn.functional as F
from src.models import Detector, DINOv2, remap_weight
from sklearn.manifold import TSNE
import torchvision.transforms as T
from matplotlib import pyplot as plt
from accelerate import Accelerator
from yacs.config import CfgNode as CN
from main import get_config, init_accelerator, set_seed, FFPP, CDF, DFDC
# logging.basicConfig(level="DEBUG")


class Obj:
    pass

In [ ]:
# fetch frames for each df_type
def generate_sample(num_samples_per_df_type=100, sample_frame_indices=[0, 1, 2], clip_duration=8, clip_frames=4, folder="frames", backbone="clip"):
    df_type_list = ["REAL", "NT", "DF", "FS", "F2F"]
    c = FFPP.get_default_config()
    c.augmentation = "normal+frame"
    c.compressions = ["c23"]
    c.types = []

    accelerator = Accelerator(mixed_precision='no')
    if backbone == "clip":
        model = clip.load("ViT-B/16")[0].visual.float()
        model.eval()

        transform = T.Compose([
            T.Resize(model.input_resolution, interpolation=T.InterpolationMode.BICUBIC),
            T.CenterCrop(model.input_resolution),
            T.ConvertImageDtype(torch.float32),
            T.Normalize((0.48145466, 0.4578275, 0.40821073),
                        (0.26862954, 0.26130258, 0.27577711)),
        ])
    elif backbone == "dino":
        model = DINOv2()
        model.eval()

        transform = T.Compose([
            T.Resize(model.input_resolution, interpolation=T.InterpolationMode.BICUBIC),
            T.CenterCrop(model.input_resolution),
            T.ConvertImageDtype(torch.float32),
            T.Normalize(
                (0.485, 0.456, 0.406),
                (0.229, 0.224, 0.225)
            ),
        ])

    for df_type in df_type_list:
        base_dir = f"./misc/{folder}/{df_type}/"
        os.makedirs(base_dir, exist_ok=True)
        c.types = [df_type]
        x = FFPP(c.clone(), clip_frames, clip_duration, transform, accelerator, split="train")
        while (int(num_samples_per_df_type * len(sample_frame_indices)) > len(glob(os.path.join(base_dir, "*")))):
            c_idx = random.randrange(0, len(x))
            frames = x[c_idx][0]["c23"]
            for i in sample_frame_indices:
                torch.save(frames[i], os.path.join(base_dir, f"{c_idx}_{str(i).zfill(3)}.pt"))


# fetch CDF/DFDC dataset frames for each label
def generate_cross_sample(num_samples_per_df_type=100, sample_frame_indices=[0, 1, 2], clip_duration=8, clip_frames=4, folder="frames", backbone="clip"):
    for dataset_cls in [DFDC, CDF]:
        c = dataset_cls.get_default_config()

        accelerator = Accelerator(mixed_precision='no')
        if backbone == "clip":
            model = clip.load("ViT-B/16")[0].visual.float()
            model.eval()

            transform = T.Compose([
                T.Resize(model.input_resolution, interpolation=T.InterpolationMode.BICUBIC),
                T.CenterCrop(model.input_resolution),
                T.ConvertImageDtype(torch.float32),
                T.Normalize((0.48145466, 0.4578275, 0.40821073),
                            (0.26862954, 0.26130258, 0.27577711)),
            ])
        elif backbone == "dino":
            model = DINOv2()
            model.eval()

            transform = T.Compose([
                T.Resize(model.input_resolution, interpolation=T.InterpolationMode.BICUBIC),
                T.CenterCrop(model.input_resolution),
                T.ConvertImageDtype(torch.float32),
                T.Normalize(
                    (0.485, 0.456, 0.406),
                    (0.229, 0.224, 0.225)
                ),
            ])

        for label in ["REAL", "FAKE"]:
            base_dir = f"./misc/{folder}/{dataset_cls.__name__}_{label}/"
            os.makedirs(base_dir, exist_ok=True)
            x = dataset_cls(c.clone(), clip_frames, clip_duration, transform, accelerator, split="test")
            while (int(num_samples_per_df_type * len(sample_frame_indices)) > len(glob(os.path.join(base_dir, "*")))):
                c_idx = random.randrange(0, len(x))
                if not x.video_info(c_idx)[1] == label:
                    continue
                frames = x[c_idx][0]
                for i in sample_frame_indices:
                    torch.save(frames[i], os.path.join(base_dir, f"{c_idx}_{str(i).zfill(3)}.pt"))

# generate_sample(100,[0,1,2],1,20,"extern/dino_1s20f/","dino")
# generate_cross_sample(100,[0,1,2],8,4,"extern/clip/8s4f/frames/","clip")

In [ ]:
@torch.no_grad()
def generate_kvs(
    glob_exp="*/*",
    folder="features",
    frame_dir="frames",
    backbone="clip",
    vpt_encoder="logs/test/magic-hill-770/best_weights.pt"
):
    if (backbone == "clip"):
        model = clip.load("ViT-B/16")[0].visual.float()
        model.load_state_dict({
            k[8:]: v for k, v in torch.load(vpt_encoder, "cpu").items() if "encoder" == k[:7]
        })
    elif (backbone == "dino"):
        assert False, "DINO is not ready for the VPT."
        model = DINOv2()
    model.eval()
    model.to("cuda")

    base_dir = f"./misc/{folder}/"
    for file in tqdm(glob(os.path.join(base_dir.replace(folder, frame_dir), glob_exp))):
        target = file.replace(frame_dir, folder)
        if (os.path.exists(target)):
            continue
        os.makedirs(os.path.split(target)[0], exist_ok=True)
        frame = torch.load(file, "cpu")
        if backbone == "clip":
            kvs = model(frame.unsqueeze(0).to("cuda"), with_q=True, with_out=True)
        elif backbone == "dino":
            kvs = model(frame.unsqueeze(0).to("cuda"), feat_keys=["q", "k", "v", "out"])
        torch.save(kvs, target)
        del kvs


# generate_kvs(
#     glob_exp="*/*",
#     folder="extern/dino_1s20f/features/",
#     frame_dir="extern/dino_1s20f/frames/",
#     backbone="dino"
# )
# generate_kvs(
#     glob_exp="*/*",
#     folder="extern/clip_vpt(50,all)/1s20f/features/",
#     frame_dir="extern/clip/1s20f/frames/",
#     backbone="clip"
# )

In [ ]:
def tsne(embeddings, labels, perplexities, graph_title, save_path, save=False):
    X = np.array(embeddings)
    plt.figure(figsize=(5 * len(perplexities), 4), layout="constrained")
    plt.suptitle(graph_title)
    for i, perplexity in enumerate(perplexities):
        print(f"TSNE Running(Perplexity:{perplexity})....")
        tsne = TSNE(n_components=2, n_iter=10000, learning_rate='auto', init='random', perplexity=perplexity)
        _X = tsne.fit_transform(X)
        print(f"TSNE Completed.")

        color = {
            "REAL": "green",
            "DF": "blue",
            "FS": "purple",
            "F2F": "darkorange",
            "NT": "red",
            "CDF_REAL": "turquoise",
            "CDF_FAKE": "deeppink",
            "DFDC_REAL": "forestgreen",
            "DFDC_FAKE": "steelblue"
        }

        plt.subplot(1, len(perplexities), i + 1)
        plt.title(f"Perplexity:{perplexity}")
        plt.gca().set_xticks([])
        plt.gca().set_yticks([])

        offset = 0
        for category_type, num in labels:
            print(category_type, num, offset)
            plt.scatter(
                _X[offset:offset + num, 0],
                _X[offset:offset + num, 1],
                3,
                # color="green" if category_type == "REAL" else "red",
                color=color[category_type],
                label=category_type if i == 0 else "",
                alpha=0.5
            )
            offset += num

        print("Rationality Check:", offset == len(_X))

    plt.gcf().legend(loc='outside right center')
    if (save):
        folder, file = os.path.split(save_path)
        if (not os.path.exists(folder)):
            os.makedirs(folder, exist_ok=True)
        plt.savefig(save_path)
    else:
        plt.show()
    plt.close()

In [ ]:
def get_detector(
        path,
        weight_name="best_weights.pt"
):
    cfg_path = path
    with open(cfg_path) as f:
        preset = CN(yaml.safe_load(f))

    mc = Detector.get_default_config().merge_from_other_cfg(preset.model)

    accelerator = Accelerator(mixed_precision=preset.system.mixed_precision)
    model = Detector(mc, preset.data.num_frames, accelerator)
    weights = remap_weight(
        model,
        torch.load(
            os.path.join(os.path.split(cfg_path)[0], weight_name),
            map_location="cpu"
        )
    )

    model.load_state_dict(
        weights
    )

    model.eval()
    model = model.to(accelerator.device)
    return model, accelerator

In [ ]:
def tsne_single(
    feature_folder,
    ratio=1.0,
    perplexities=[20, 50, 80],
    patch_num=14,
    save=False,
    save_folder="./misc/graphs/test/",
    glob_exp="*_000.*",
    num_frames_per_sample=1
):

    category_type_list = [d for d in os.listdir(feature_folder) if os.path.isdir(os.path.join(feature_folder, d))]

    for target_layer in range(0, 12):
        print(f"Processing  Layer {target_layer}")

        layer_features = {
            i: {t: [] for t in ["q", "out", "k", "v"]}
            for i in category_type_list
        }

        for df_type in layer_features.keys():
            base_dir = os.path.join(feature_folder, df_type)
            files = sorted(glob(os.path.join(base_dir, glob_exp)))
            files = files[:int(len(files) * ratio // num_frames_per_sample * num_frames_per_sample)]

            print("Files:", files)
            print("Length:", len(files))

            for file in files:
                features = torch.load(file, "cpu")[target_layer]
                for subject in layer_features[df_type].keys():
                    # cls only
                    # print(features[subject][:,0].shape)
                    # return
                    # layer_features[df_type][subject] += features[subject][:, 0].view((-1, 768)).half().tolist()
                    # without cls
                    # layer_features[df_type][subject] += features[subject][:,1:].view((-1, 768)).half().tolist()
                    # all
                    # layer_features[df_type][subject] += features[subject].view((-1, 768)).half().tolist()
                    # crop center
                    # layer_features[df_type][subject] += features[subject][:,1:].view((-1,14,14,768))[:,4:11,4:11,:].reshape((-1,768)).half().tolist()
                    # flatten
                    # layer_features[df_type][subject] += features[subject].half().view((1,-1)).tolist()
                    # crop center flatten
                    qaurt_patch = patch_num // 4
                    layer_features[df_type][subject].append(features[subject][:, 1:].view(
                        (-1, patch_num, patch_num, 768))[:, qaurt_patch:-qaurt_patch, qaurt_patch:-qaurt_patch, :].flatten().tolist()
                    )

        print(f"Layer {target_layer} Features Loaded.")
        for subject in ["q", "out", "k", "v"]:
            print(f"Processing Subject {subject}.")
            labels = []
            embeddings = []
            for df_type in layer_features.keys():
                features = layer_features[df_type][subject]
                labels.append((df_type, len(features)))
                embeddings += features

            print(f"Embeddings Built.")
            tsne(
                embeddings,
                labels,
                perplexities,
                f"Layer={target_layer},Subject={subject},Ratio={ratio}",
                os.path.join(save_folder, f"L{target_layer}S{subject}.pdf"),
                save
            )


tsne_single("./misc/extern/clip/vpt/1s20f/features/", save=True, save_folder="./misc/graphs/vpt_1s20f_single_ccrop/")

In [ ]:
# CLIP EXCLUSIVE
# def tsne_adapter():
#     torch.cuda.empty_cache()
#     ratio = 1.0
#     glob_exp = "*_0.*"
#     image_folder = "./misc/graphs/test/"
#     feature_folder = "./misc/1s20f/features/"
#     num_frames_per_sample = 1
#     perplexities = [20, 50, 80]

#     os.makedirs(image_folder, exist_ok=True)

#     df_type_list = ["REAL", "DF"]
#     subjects = ["k", "v"]
#     layer_features = [
#         {
#             i: {t: [] for t in ["k", "v"]}
#             for i in df_type_list
#         }
#         for _ in range(6)
#     ]

#     # load adapter from model
#     model, _ = get_detector()
#     adapter = model.adapter

#     for df_type in df_type_list:
#         base_dir = os.path.join(feature_folder, df_type)
#         files = sorted(glob(os.path.join(base_dir, glob_exp)))
#         files = files[:int(len(files) * ratio // num_frames_per_sample * num_frames_per_sample)]

#         print("Files:", files)
#         print("Length:", len(files))

#         for file in files:
#             features = torch.load(file, "cpu")[6:]

#             # drop unrequired features
#             for i in range(len(features)):
#                 features[i].pop("q")
#                 features[i].pop("out")
#                 features[i]["v"] = features[i]["v"][:, 1:].view(1, 1, -1, 12, 64).to("cuda")
#                 features[i]["k"] = features[i]["k"][:, 1:].view(1, 1, -1, 12, 64).to("cuda")

#             # adapt features
#             features = adapter(features)

#             # save adapted features
#             for i in range(len(features)):
#                 for subject in subjects:
#                     layer_features[i][df_type][subject].append(
#                         features[i][subject].flatten().tolist()
#                     )

#     for target_layer in range(6):
#         for subject in subjects:
#             print(f"Processing Subject {subject}.")
#             labels = []
#             embeddings = []
#             for df_type in df_type_list:
#                 features = layer_features[target_layer][df_type][subject]
#                 labels.append((df_type, len(features)))
#                 embeddings += features

#             print(f"Embeddings Built.")
#             tsne(
#                 embeddings,
#                 labels,
#                 perplexities,
#                 f"Layer={target_layer},Subject={subject},Ratio={ratio}",
#                 os.path.join(image_folder, f"L{target_layer}S{subject}.pdf")
#             )

In [ ]:
def tsne_temporal(
    feature_folder,
    ratio=1.0,
    temporal_indices=[0, 1],
    alignment=False,
    softmax_first=False,
    spatial_size=7,
    perplexities=[20, 50, 80],
    patch_num=14,
    save=False,
    save_folder="./misc/graphs/test/",
    glob_exp="*",
    num_frames_per_sample=3
):
    quart_patch = patch_num // 4
    assert spatial_size % 2 == 1

    category_type_list = [d for d in os.listdir(feature_folder) if os.path.isdir(os.path.join(feature_folder, d))]

    for target_layer in range(0, 12):
        print(f"Processing  Layer {target_layer}")

        layer_features = {
            i: {"attn": []}
            for i in category_type_list
        }

        for df_type in layer_features.keys():
            base_dir = os.path.join(feature_folder, df_type)
            files = sorted(glob(os.path.join(base_dir, glob_exp)))
            files = files[:int(len(files) * ratio // num_frames_per_sample * num_frames_per_sample)]

            print("Files:", files)
            print("Length:", len(files))

            for offset in range(0, len(files), num_frames_per_sample):

                # fetch consecutive frames
                vid_features = {"k": [], "q": []}
                for temporal_index in temporal_indices:
                    file = files[offset + temporal_index]
                    features = torch.load(file, "cpu")[target_layer]
                    for subject in vid_features.keys():
                        vid_features[subject].append(features[subject][:, 1:].view((-1, 768)))

                # stack consecutive accroding to the subject
                for k in vid_features.keys():
                    vid_features[k] = torch.stack(vid_features[k])

                # perform cross-frame patch attention, reshape the attention scores into spatial form.
                aff = torch.einsum(
                    'qhd,khd->hqk',
                    vid_features["q"][0].unflatten(-1, (12, 64)) / (vid_features["q"].shape[-1]**0.5),
                    vid_features["k"][-1].unflatten(-1, (12, 64))
                )

                if (softmax_first):
                    aff = aff.softmax(dim=-1)

                aff = aff.view((-1, patch_num, patch_num, patch_num, patch_num))

                # fetch neighbor patch attention score of proximity frames
                patch_features = []
                for j in range(quart_patch, patch_num - quart_patch):
                    for k in range(quart_patch, patch_num - quart_patch):

                        if (alignment):
                            half_size = spatial_size // 2
                            att_feat = aff[:, j, k, j - half_size:j + half_size +
                                           1, k - half_size:k + half_size + 1].flatten(1)
                        else:
                            att_feat = aff[:, j, k,].flatten(1)

                        if not softmax_first:
                            att_feat = att_feat.softmax(dim=-1)

                        att_feat = att_feat.flatten().tolist()
                        patch_features += att_feat

                layer_features[df_type]["attn"].append(patch_features)

        print(f"Layer {target_layer} Features Loaded.")
        for subject in ["attn"]:
            print(f"Processing Subject {subject}.")
            labels = []
            embeddings = []
            for df_type in layer_features.keys():
                features = layer_features[df_type][subject]
                labels.append((df_type, len(features)))
                embeddings += features
            print(f"Embeddings Built.")

            tsne(
                embeddings,
                labels,
                perplexities,
                f"Layer={target_layer},Subject={subject},Ratio={ratio}",
                os.path.join(save_folder, f"L{target_layer}S{subject}.pdf"),
                save
            )

            print(f"Subject {subject} completed.")

# tsne_temporal("./misc/extern/dino_1s20f/features/",patch_num=16)
# tsne_temporal(
#     "./misc/extern/clip/1s20f/features/",
#     alignment=True,
#     softmax_first=True,
#     save=True,
#     save_folder="./misc/graphs/1s20f_temporal_ccrop/"
# )

In [ ]:
def tsne_spatial(
    feature_folder,
    ratio=1.0,
    normalize=False,
    perplexities=[20, 50, 80],
    patch_num=14,
    save=False,
    save_folder="./misc/graphs/test/",
    glob_exp="*_000.*",
    num_frames_per_sample=1
):
    category_type_list = [d for d in os.listdir(feature_folder) if os.path.isdir(os.path.join(feature_folder, d))]

    for target_layer in range(0, 12):
        print(f"Processing  Layer {target_layer}")

        layer_features = {
            i: {t: [] for t in ["attn"]}
            for i in category_type_list
        }

        for df_type in layer_features.keys():
            base_dir = os.path.join(feature_folder, df_type)
            files = sorted(glob(os.path.join(base_dir, glob_exp)))
            files = files[:int(len(files) * ratio // num_frames_per_sample * num_frames_per_sample)]

            print("Files:", files)
            print("Length:", len(files))

            for file in files:
                features = torch.load(file, "cpu")[target_layer]
                frame_features = {"q": None, "k": None}
                for subject in frame_features.keys():
                    frame_features[subject] = features[subject][:, 1:].view((-1, 768))

                aff = torch.einsum(
                    'qh,kh->qk',
                    frame_features["q"] / (frame_features["q"].shape[-1]**0.5),
                    frame_features["k"]
                )

                if (normalize):
                    aff = aff.softmax(dim=-1)

                quart_patch = patch_num // 4
                layer_features[df_type]["attn"].append(
                    aff.view(patch_num, patch_num, -1)[quart_patch:-quart_patch,
                                                       quart_patch:-quart_patch].flatten().tolist()
                )

        print(f"Layer {target_layer} Features Loaded.")
        for subject in ["attn"]:
            print(f"Processing Subject {subject}.")
            labels = []
            embeddings = []
            for df_type in layer_features.keys():
                features = layer_features[df_type][subject]
                labels.append((df_type, len(features)))
                embeddings += features

            print(f"Embeddings Built.")

            tsne(
                embeddings,
                labels,
                perplexities,
                f"Layer={target_layer},Subject={subject},Ratio={ratio}",
                os.path.join(save_folder, f"L{target_layer}S{subject}.pdf"),
                save
            )

            print(f"Subject {subject} completed.")


# tsne_spatial("./misc/extern/dino_1s20f/features/", patch_num=16)
# tsne_spatial(
#     "./misc/extern/clip/1s20f/features/",
#     normalize=True,
#     save=True,
#     save_folder="./misc/graphs/1s20f_spatial_ccrop2/"
# )

In [ ]:
# def tsne_semantic():
#     import pickle
#     torch.cuda.empty_cache()

#     # load semantic features
#     with open("misc/semantic_patches.pickle", "rb") as f:
#         semantic_queries = pickle.load(f)
#     num_parts = len(semantic_queries["q"].keys())
#     ratio = 1.0
#     glob_exp = "*_0.*"
#     image_folder = "./misc/graphs/test/"
#     feature_folder = "./misc/1s20f/features/"
#     num_frames_per_sample = 1
#     perplexities = [20, 50, 80]

#     # vert_filter = torch.tensor([[[
#     #     [-1, -2, -1],
#     #     [0, 0, 0],
#     #     [1, 2, 1]
#     # ]]]).float().repeat(num_parts, 1, 1, 1)

#     # hori_filter = torch.tensor([[[
#     #     [-1, 0, 1],
#     #     [-2, 0, 2],
#     #     [-1, 0, 1]
#     # ]]]).float().repeat(num_parts, 1, 1, 1)

#     os.makedirs(image_folder, exist_ok=True)

#     patch_loc_embedding = torch.stack(
#         [torch.randn(10) + 1 * i for i in range(196)]
#     )

#     for target_layer in range(0, 12):
#         print(f"Processing  Layer {target_layer}")

#         layer_features = {
#             i: {t: [] for t in ["attn"]}
#             for i in df_type_list
#         }

#         for df_type in layer_features.keys():
#             base_dir = os.path.join(feature_folder, df_type)
#             files = sorted(glob(os.path.join(base_dir, glob_exp)))
#             files = files[:int(len(files) * ratio // num_frames_per_sample * num_frames_per_sample)]

#             print("Files:", files)
#             print("Length:", len(files))

#             for file in files:
#                 features = torch.load(file, "cpu")[target_layer]
#                 frame_features = {"q": None, "k": None, "v": None, "out": None}
#                 for subject in frame_features.keys():
#                     frame_features[subject] = features[subject][:, 1:].view((-1, 768))

#                 # semantic_layer_queries = torch.stack([
#                 #     semantic_queries["q"][part][target_layer] for part in semantic_queries["q"].keys()
#                 # ]).unflatten(-1,(12,64))

#                 # aff = torch.einsum(
#                 #     'qhd,khd->qkh',
#                 #     # semantic_layer_queries/(semantic_layer_queries.shape[-1]**0.5),
#                 #     semantic_layer_queries,
#                 #     frame_features["k"].unflatten(-1,(12,64))
#                 # )

#                 semantic_layer_queries = torch.stack([
#                     semantic_queries["q"][part][target_layer] for part in semantic_queries["q"].keys()
#                 ])
#                 ########################
#                 # aff = torch.einsum(
#                 #     'qh,kh->qk',
#                 #     semantic_layer_queries/(semantic_layer_queries.shape[-1]**0.5),
#                 #     frame_features["k"]
#                 # ).softmax(dim=-1).view((1,-1,14,14))

#                 # aff = F.conv2d(aff,vert_filter, padding=1,groups=num_parts) + F.conv2d(aff,hori_filter, padding=1,groups=num_parts)
#                 ########################
#                 aff = torch.einsum(
#                     'qh,kh->qk',
#                     semantic_layer_queries / (semantic_layer_queries.shape[-1]**0.5),
#                     frame_features["k"]
#                 ).softmax(dim=-1)

#                 aff = torch.einsum(
#                     'qk,kn->qn',
#                     aff,
#                     patch_loc_embedding
#                 )

#                 # print(aff.shape)
#                 # return
#                 layer_features[df_type]["attn"].append(aff.flatten().tolist())

#         print(f"Layer {target_layer} Features Loaded.")

#         for subject in ["attn"]:
#             print(f"Processing Subject {subject}.")
#             labels = []
#             embeddings = []
#             for df_type in layer_features.keys():
#                 features = layer_features[df_type][subject]
#                 labels.append((df_type, len(features)))
#                 embeddings += features
#             print(f"Embeddings Built.")

#             tsne(
#                 embeddings,
#                 labels,
#                 perplexities,
#                 f"Layer={target_layer},Subject={subject},Ratio={ratio}",
#                 os.path.join(image_folder, f"L{target_layer}S{subject}.pdf")
#             )

#             print(f"Subject {subject} completed.")

In [ ]:
@torch.inference_mode()
def tsne_decoder(
    model_name,
    model_config,
    num_samples_per_df=50,
    perplexities=[20, 50, 80],
    save=False,
    save_folder="./misc/graphs/test/"
):
    torch.cuda.empty_cache()

    model, accelerator = get_detector(model_config)

    transform = T.Compose([
        T.Resize(model.encoder.input_resolution, interpolation=T.InterpolationMode.BICUBIC),
        T.CenterCrop(model.encoder.input_resolution),
        T.ConvertImageDtype(torch.float32),
        T.Normalize((0.48145466, 0.4578275, 0.40821073),
                    (0.26862954, 0.26130258, 0.27577711)),
    ])

    df_type_list = ["REAL", "DF", "NT", "FS", "F2F"]
    df_features = {df_type: [] for df_type in df_type_list}
    for df_type in df_type_list:
        c = FFPP.get_default_config()
        c.augmentation = "normal+frame"
        c.compressions = ["c23"]
        c.types = [df_type]

        x = FFPP(c.clone(), 20, 4, transform, accelerator, split="test")
        indices = [i for i in range(len(x))]
        random.shuffle(indices)
        for i in tqdm(indices[:num_samples_per_df]):
            data = x[i]
            task_logits, other_features = model.predict(
                data[0]["c23"].unsqueeze(0).to("cuda"),
                data[2].unsqueeze(0).to("cuda"),
                with_video_features=True
            )
            df_features[df_type].append(other_features["video"].flatten().tolist())

    # with CDF & DFDC
    df_features["CDF_REAL"] = []
    df_features["DFDC_REAL"] = []
    df_features["CDF_FAKE"] = []
    df_features["DFDC_FAKE"] = []
    for dataset_cls in [CDF, DFDC]:
        c = dataset_cls.get_default_config()
        x = dataset_cls(c.clone(), 20, 4, transform, accelerator, split="test")
        for label in ["REAL", "FAKE"]:
            indices = [i for i in range(len(x))]
            indices = [i for i in indices if x.video_info(i)[1] == label]
            random.shuffle(indices)
            for i in tqdm(indices[:num_samples_per_df]):
                data = x[i]
                task_logits, other_features = model.predict(
                    data[0].unsqueeze(0).to("cuda"),
                    data[2].unsqueeze(0).to("cuda"),
                    with_video_features=True
                )
                df_features[f"{dataset_cls.__name__}_{label}"].append(other_features["video"].flatten().tolist())

    labels = []
    embeddings = []
    for df_type in df_features.keys():
        features = df_features[df_type]
        labels.append((df_type, len(features)))
        embeddings += features

    tsne(
        embeddings,
        labels,
        perplexities,
        f"Categories for '{model_name}'",
        os.path.join(save_folder, f"{model_name}_category_tsne.pdf"),
        save
    )


# tsne_decoder(50,perplexities=[20,35,50],detector_path="logs/deepfake/test/")
for name, path in [
    # ("BEST","logs/deepfake/deepfake/best/new.yaml"),
    # ("FARL+tune_all","logs/test/misunderstood-glitter-939/setting.yaml"),
    # ("JULY(-adapt)+tune_all", "logs/test/sage-totem-907/setting.yaml"),
    # ("JULY+tune_all", "logs/test/JULY+tune_all/setting.yaml")
    # ("LOO(NT)", "logs/test/sparkling-firebrand-1108/setting.yaml"),
    ("2Query", "logs/test/silver-gorge-1037/setting.yaml")
]:
    tsne_decoder(
        name,
        path,
        50,
        perplexities=[20, 35, 50],
        save=True
    )